In [ ]:
%run /edfi_fetch_urls

In [1]:
# workspace = "dev"
# apiVersion = "5.2"
# moduleName = "Ed-Fi10"
# schoolYear = None
# districtId = None

In [63]:
%run /edfi_py

In [46]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"

parameterized = False

In [47]:
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/etl_entities/current_run_data'

    _, edfiEntities = edfi.listSpecifiedEntities(edfiEntitiesPath)
else:
    edfiEntities = "All"  
tpmEntities = edfiEntities

In [89]:
def ingest_edfi_dataset(tables_source, 
                        items = 'All'):
    full_source_path = f"stage1/Transactional/{tables_source}"
    if items == 'All':
        items = oea.get_folders(full_source_path) 
   
    options = {'format': 'json', 'multiline': False}
    for item in items: 
        table_path = full_source_path +'/'+ item
        try:
            entity_path = f"{tables_source}/{item}"
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                oea.path_exists(oea.to_url(entity_path)):
                    oea.ingest(entity_path, 'id', options)
        except AnalysisException as e:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            pass

In [90]:
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi',
                    edfiEntities)
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm',
                    tpdmEntities)